In [17]:
# Install missing packages (run once in this environment)
%pip install statsmodels
%pip install pingouin

# Import relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.miscmodels.ordinal_model import OrderedModel
import pingouin as pg
from scipy import stats

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ------------ --------------------------- 2.6/8.7 MB 13.1 MB/s eta 0:00:01
   ------------------------ --------------- 5.2/8.7 MB 12.9 MB/s eta 0:00:01
   ------------------------------------ --- 7.9/8.7 MB 12.9 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 11.8 MB/s  0:00:00
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 9.6 MB/s  0:00:00

   ---------------------------------------- 0/7 [threadpoolctl]
   ----- ---------------------------------- 1/7 [tabulate]
   ----- ---------------------------------- 1/7 [tabulate]
   ----- ---------------------------------- 1/7 [tabulate]
   ----- ---------------------------------- 1/7 [tabulate]
   ----------- ---------------------------- 2/7 [joblib]
   ----------- ---------------------------- 2/7 

In [ ]:
# Load the df_chat_surveys.csv file
data_dir = Path('chat_exports')
df_chat_surveys = pd.read_csv(data_dir / 'df_chat_surveys_standardized.csv', sep=',')
print(f"df_chat_surveys shape: {df_chat_surveys.shape}")
# Preview the dataset
df_chat_surveys.head()

df_chat_surveys shape: (45, 74)


,participant_id,created_at,ip_address,device_type,background_submitted_at,q1_age_group,q2_gender,q3_education,q4_employment,q5_nationality,...,q29_tool_easier_task,q30_tool_useful,q31_tool_too_much_info,q32_tool_difficult_focus,q33_results_accurate,q34_results_trust,q35_results_complete,q36_results_relevant,q37_results_useful,q40_purchase_likelihood
0,fe4a79bb-642e-49d8-920a-a5f39035ffe3,2025-11-20 16:22:11.281835+00,"84.17.45.206,84.17.45.206, 99.82.172.145",desktop,2025-11-20 16:23:00.392222+00:00,18-24,Female,High school or below,Student,Indo,...,6,5,7,6,7,5,5,6,7,2
1,151939c4-8aae-490b-92eb-c7d353a4a95c,2025-11-18 18:15:57.927115+00,"193.197.8.143,193.197.8.143, 99.82.163.77",desktop,2025-11-18 18:17:10.869409+00:00,35-44,Male,Master's degree,Employed,Iranian,...,6,7,6,4,4,5,5,5,5,6
2,e2f70344-8588-468c-9dc2-8c2993f5b6c9,2025-11-15 23:28:20.372283+00,"178.84.99.93,178.84.99.93, 3.2.59.182",desktop,2025-11-15 23:29:47.088239+00:00,18-24,Female,Bachelor's degree,Student,Nederland,...,5,5,6,5,3,4,4,6,3,5
3,88584cfe-a550-4209-a1f3-be628bb2647b,2025-11-13 16:29:27.470605+00,"89.92.129.28,89.92.129.28, 99.82.161.171",desktop,2025-11-13 16:30:20.934922+00:00,35-44,Male,Master's degree,Self-employed,French,...,6,6,6,4,5,5,5,5,5,4
4,25e47297-047a-4582-bc17-c13f5b2ef07b,2025-11-13 12:54:47.011689+00,"153.92.90.3,153.92.90.3, 99.82.163.72",mobile,2025-11-13 12:55:37.488309+00:00,18-24,Female,Bachelor's degree,Student,Georgian,...,5,5,4,5,7,7,7,7,7,4


In [ ]:
# Load the df_gg_surveys.csv file
data_dir = Path('google_exports')
df_gg_surveys = pd.read_csv(data_dir / 'df_gg_surveys_standardised.csv', sep=',')
print(f"df_gg_surveys shape: {df_gg_surveys.shape}")
# Preview the dataset
df_gg_surveys.head()

df_gg_surveys shape: (52, 69)


,participant_id,q1_age_group,q2_gender,q3_education,q4_employment_status,q5_nationality,q5_nationality_sd,q6_residence,q6_residence_sd,q7_chatbot_familiarity,...,q42_comments,background_submitted_at,instruction_submitted_at,postsurvey_submitted_at,ip_address,device_type,results_submitted_at,session_start_time,session_end_time,session_duration_ms
0,09e6255b-ed66-4eea-a314-104a08130ac0,35-44,Female,Doctorate / PhD,Employed,UK,UK,UK,UK,7,...,"I liked the more minimal interface, and I like...",2025-10-31 22:11:56.864775+00:00,2025-10-31 22:12:41.977243+00:00,2025-10-31 22:22:26.969691+00:00,129.67.117.187,desktop,2025-10-31 22:19:44.042285+00:00,2025-10-31 22:10:58.834+00,2025-10-31 22:18:19.43+00,440596.0
1,15ef74b6-a61a-474c-b855-696b20ce58fb,55 and above,Male,Master's degree,Employed,German,Germany,Germany,Germany,1,...,Maybe,2025-10-24 14:12:35.527412+00:00,2025-10-24 14:21:30.862457+00:00,2025-10-24 14:28:43.167002+00:00,213.146.69.174,desktop,2025-10-24 14:27:11.483904+00:00,2025-10-24 14:11:51.631+00,2025-10-24 14:24:48.07+00,776439.0
2,1f0df1be-a1ea-4080-90c3-230fe9e35174,18-24,Female,Bachelor's degree,Student,United States,USA,United States,USA,6,...,"Yes, it showed me relevant results for what I ...",2025-11-10 03:25:51.855943+00:00,2025-11-10 03:26:25.769685+00:00,2025-11-10 03:29:58.830293+00:00,108.2.105.241,desktop,2025-11-10 03:28:20.935272+00:00,2025-11-10 03:25:14.283+00,2025-11-10 03:26:06.698+00,52415.0
3,2afa9961-1844-49e8-80fc-444466532f46,25-34,Female,Master's degree,Employed,Russian,NaN,Russia,Russia,6,...,"Yes, the tool was transparent and easy to use,...",2025-10-24 17:08:04.105300+00:00,2025-10-24 17:10:42.739044+00:00,2025-10-24 17:15:27.907607+00:00,2a00:1370:8180:f390:10f5:bcf9:8f0:be3f,mobile,2025-10-24 17:12:41.450159+00:00,2025-10-24 17:06:39.799+00,2025-10-24 17:10:49.029+00,249230.0
4,2d8dd1db-9d38-49e0-bf03-5b4735523d27,18-24,Female,High school or below,Student,Swiss,Switzerland,Switzerland,Switzerland,7,...,"No, AI didn’t show the classical bar abovd",2025-10-29 09:54:21.050268+00:00,2025-10-29 09:54:41.702835+00:00,2025-10-29 10:01:23.058400+00:00,172.225.188.246,mobile,2025-10-29 09:58:09.817211+00:00,2025-10-29 09:52:50.793+00,2025-10-29 09:54:46.465+00,115672.0


In [ ]:
df_chat_surveys.columns

Index(['participant_id', 'created_at', 'ip_address', 'device_type',
       'background_submitted_at', 'q1_age_group', 'q2_gender', 'q3_education',
       'q4_employment', 'q5_nationality', 'q5_nationality_sd', 'q6_residence',
       'q6_residence_sd', 'q7_chatbot_familiarity', 'q8_data_quality',
       'q9_chatbot_usage', 'q10_search_tools', 'q10_tool_1', 'q10_tool_2',
       'q10_tool_3', 'q10_tool_4', 'q11_budget', 'instruction_submitted_at',
       'results_submitted_at', 'q12_brand_model', 'q12_brand_model_1_sd',
       'q12_brand_model_2_sd', 'q12_brand_only', 'q13_storage',
       'q13_storage_1_sd', 'q13_storage_2_sd', 'q14_color', 'q14_color_1_sd',
       'q14_color_2_sd', 'q14_color_3_sd', 'q14_color_4_sd', 'q14_color_5_sd',
       'q15_price', 'q15_price_euro', 'q15_price_range', 'q16_website',
       'q17_price_importance', 'q18_important_features', 'q18_feature_1',
       'q18_feature_2', 'q18_feature_3', 'postsurvey_submitted_at',
       'q38_attention', 'q41_duration', 'q

In [ ]:
df_gg_surveys.columns

Index(['participant_id', 'q1_age_group', 'q2_gender', 'q3_education',
       'q4_employment_status', 'q5_nationality', 'q5_nationality_sd',
       'q6_residence', 'q6_residence_sd', 'q7_chatbot_familiarity',
       'q8_data_quality', 'q9_ai_usage_frequency', 'q10_search_tools',
       'q10_tool_1', 'q10_tool_2', 'q10_tool_3', 'q10_tool_4', 'q11_budget',
       'q12_brand_model', 'q12_brand_only', 'q13_storage_capacity',
       'q13_storage_1_sd', 'q13_storage_2_sd', 'q14_color', 'q14_color_1_sd',
       'q15_price', 'q15_price_euro', 'q15_price_range', 'q16_website',
       'q17_price_importance', 'q18_important_features', 'q18_feature_1',
       'q18_feature_2', 'q18_feature_3', 'q19_task_easy', 'q20_task_quick',
       'q21_task_familiar', 'q22_tool_reliable', 'q23_tool_practical',
       'q24_tool_like', 'q25_tool_easy_use', 'q26_tool_clear',
       'q27_tool_control', 'q28_tool_info_provided', 'q29_tool_easier_task',
       'q30_tool_useful', 'q31_tool_too_much_info', 'q32_tool_dif

## RQ3: Comparing User´s Subjective Experience During the Search

### 3.1 Attitude towards the Tool (Questions 22-24)

In [26]:
# -------------------------------------------------------
# 1. LOAD DATA
# -------------------------------------------------------
# df_gg_surveys = pd.read_csv("/mnt/data/df_gg_surveys_standardised.csv")
# df_chat_surveys = pd.read_csv("/mnt/data/df_chat_surveys_standardized.csv")

# -------------------------------------------------------
# 2. ADD CONDITION VARIABLE
# Google = 0, Perplexity = 1
# -------------------------------------------------------
df_gg_surveys["condition"] = 0
df_chat_surveys["condition"] = 1

# -------------------------------------------------------
# 3. MERGE DATASETS
# -------------------------------------------------------
df = pd.concat([df_gg_surveys, df_chat_surveys], ignore_index=True)

# The three items used for the multi-item scale
items_22_24 = ["q22_tool_reliable", "q23_tool_practical", "q24_tool_like"]
cols_25_27 = ["condition"] + items_22_24

df_scale = df[cols_25_27].dropna()

# -------------------------------------------------------
# 4. COMPUTE CRONBACH'S ALPHA (MANUAL FORMULA)
# -------------------------------------------------------
item_scores = df_scale[items_22_24].to_numpy()

k = item_scores.shape[1]  # number of items
item_variances = item_scores.var(axis=0, ddof=1)
total_score_variance = item_scores.sum(axis=1).var(ddof=1)

alpha = k/(k-1) * (1 - item_variances.sum()/total_score_variance)

print("\n=== Cronbach's Alpha ===")
print(alpha)

# -------------------------------------------------------
# 5. CREATE COMPOSITE SCALE (MEAN SCORE)
# -------------------------------------------------------
df_scale["tool_eval_mean"] = df_scale[items].mean(axis=1)

print("\n=== Head of Composite Scale Data ===")
print(df_scale.head())

# -------------------------------------------------------
# 6. RUN OLS REGRESSION ON THE COMPOSITE SCALE
# -------------------------------------------------------
model_scale = smf.ols("tool_eval_mean ~ condition", data=df_scale).fit()

print("\n=== OLS Regression Results (Composite Scale) ===\n")
print(model_scale.summary())

# -------------------------------------------------------
# 7. GROUP MEANS (OPTIONAL)
# -------------------------------------------------------
print("\n=== Group Means (Google vs Perplexity) ===")
print(df_scale.groupby("condition")["tool_eval_mean"].mean())



=== Cronbach's Alpha ===
0.8987032079694561

=== Head of Composite Scale Data ===
   condition  q22_tool_reliable  q23_tool_practical  q24_tool_like  \
0          0                  2                   3              4   
1          0                  4                   4              4   
2          0                  6                   7              6   
3          0                  7                   7              7   
4          0                  5                   6              7   

   tool_eval_mean  
0        3.000000  
1        4.000000  
2        6.333333  
3        7.000000  
4        6.000000  

=== OLS Regression Results (Composite Scale) ===

                            OLS Regression Results                            
Dep. Variable:         tool_eval_mean   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     3.424
Date:    

**Results on Affective and Attitude about the Tools:**
1. **Reliability (Cronbach´s alpha)**: Cronbach’s alpha was high (α ≈ 0.90), indicating that the three items (“reliable”, “practical”, “like”) form a coherent, internally consistent scale capturing participants’ overall attitude towards the tool.
2. **Difference in attitude between Google and Perplexity**: An OLS regression predicting the composite attitude score from tool condition (0 = Google, 1 = Perplexity) showed that participants tended to evaluate Google more favorably than Perplexity (b = -0.55, SE = 0.30, 95% CI [-1.14, 0.04]). However, this difference was only marginally significant and did not reach conventional levels of statistical significance (p = .067).
3. **Group means**: On the 7‑point attitude scale, participants evaluated Google more positively (M = 4.95) than Perplexity (M = 4.40) — about 0.55 points higher on average. This pattern is consistent with the negative regression coefficient, although the difference was only marginally significant (p = .067).

### 3.2 Perceived Ease of Use about the Tool (Questions 25-27)

In [20]:
# -------------------------------------------------------
# 3.2 Attitude scale based on Q25–Q27
# -------------------------------------------------------

# 1. Specify items
items_25_27 = ["q25_tool_easy_use", "q26_tool_clear", "q27_tool_control"]  # <-- replace with exact column names
cols_25_27 = ["condition"] + items_25_27

# 2. Subset and drop rows with missing values in these items
df_scale_25_27 = df[cols_25_27].dropna()

# 3. Compute Cronbach's alpha (manual)
item_scores_25_27 = df_scale_25_27[items_25_27].to_numpy()

k_25_27 = item_scores_25_27.shape[1]  # number of items (3)
item_variances_25_27 = item_scores_25_27.var(axis=0, ddof=1)
total_score_variance_25_27 = item_scores_25_27.sum(axis=1).var(ddof=1)

alpha_25_27 = k_25_27/(k_25_27 - 1) * (1 - item_variances_25_27.sum() / total_score_variance_25_27)

print("\n=== Cronbach's Alpha (Q25–Q27 scale) ===")
print(alpha_25_27)

# 4. Create composite scale (mean of Q25–Q27)
df_scale_25_27["tool_eval_q25_27_mean"] = df_scale_25_27[items_25_27].mean(axis=1)

print("\n=== Head of Composite Scale Data (Q25–Q27) ===")
print(df_scale_25_27.head())

# 5. Run OLS regression: composite ~ condition
model_scale_25_27 = smf.ols("tool_eval_q25_27_mean ~ condition", data=df_scale_25_27).fit()

print("\n=== OLS Regression Results (Q25–Q27 Composite) ===\n")
print(model_scale_25_27.summary())

# 6. Group means for Google vs Perplexity
print("\n=== Group Means (Google vs Perplexity, Q25–Q27 scale) ===")
print(df_scale_25_27.groupby("condition")["tool_eval_q25_27_mean"].mean())


=== Cronbach's Alpha (Q25–Q27 scale) ===
0.8603600675939164

=== Head of Composite Scale Data (Q25–Q27) ===
   condition  q25_tool_easy_use  q26_tool_clear  q27_tool_control  \
0          0                  4               5                 5   
1          0                  5               5                 5   
2          0                  7               6                 6   
3          0                  7               7                 7   
4          0                  7               7                 7   

   tool_eval_q25_27_mean  
0               4.666667  
1               5.000000  
2               6.333333  
3               7.000000  
4               7.000000  

=== OLS Regression Results (Q25–Q27 Composite) ===

                              OLS Regression Results                             
Dep. Variable:     tool_eval_q25_27_mean   R-squared:                       0.064
Model:                               OLS   Adj. R-squared:                  0.055
Method:        

**Results on Perceived ease of using the tool:**
1. **Reliability (Cronbach’s alpha, Q25–Q27):** The perceived ease‑of‑use scale based on Questions 25–27 (q25_tool_easy_use, q26_tool_clear, q27_tool_control) showed good internal consistency (Cronbach’s alpha = 0.86). This indicates that the three items **coherently** measure participants’ perceived ease of using the tool.

2. **Difference in perceived ease of use between Google and Perplexity:** An OLS regression predicting the composite ease‑of‑use score from tool condition (0 = Google, 1 = Perplexity) showed that participants evaluated Google as easier to use than Perplexity (b = -0.69, SE = 0.27, 95% CI [-1.22, -0.15]). This difference was **statistically significant (p = .012)**, indicating a reliable advantage for Google in perceived ease of use.

3. **Group means (Q25–Q27 scale):** On the 7‑point ease‑of‑use scale, participants rated Google more positively (M = 5.49) than Perplexity (M = 4.81), a difference of about 0.69 points on average. This pattern is consistent with the negative regression coefficient and illustrates that **Google was clearly perceived as easier to use than Perplexity.**

### 3.3 Perceived Usefulness (Questions 28-30, 'q28_tool_info_provided', 'q29_tool_easier_task','q30_tool_useful')

In [21]:
# -------------------------------------------------------
# 3.3 Perceived Usefulness (Q28–Q30)
# -------------------------------------------------------

# 1. Specify items
items_28_30 = ["q28_tool_info_provided", "q29_tool_easier_task", "q30_tool_useful"]
cols_28_30 = ["condition"] + items_28_30

# 2. Subset and drop rows with missing values in these items
df_scale_28_30 = df[cols_28_30].dropna()

# 3. Compute Cronbach's alpha (manual)
item_scores_28_30 = df_scale_28_30[items_28_30].to_numpy()

k_28_30 = item_scores_28_30.shape[1]  # number of items (3)
item_variances_28_30 = item_scores_28_30.var(axis=0, ddof=1)
total_score_variance_28_30 = item_scores_28_30.sum(axis=1).var(ddof=1)

alpha_28_30 = k_28_30/(k_28_30 - 1) * (1 - item_variances_28_30.sum() / total_score_variance_28_30)

print("\n=== Cronbach's Alpha (Q28–Q30 usefulness scale) ===")
print(alpha_28_30)

# 4. Create composite scale (mean of Q28–Q30)
df_scale_28_30["tool_eval_q28_30_mean"] = df_scale_28_30[items_28_30].mean(axis=1)

print("\n=== Head of Composite Scale Data (Q28–Q30) ===")
print(df_scale_28_30.head())

# 5. Run OLS regression: composite ~ condition
model_scale_28_30 = smf.ols("tool_eval_q28_30_mean ~ condition", data=df_scale_28_30).fit()

print("\n=== OLS Regression Results (Q28–Q30 Composite) ===\n")
print(model_scale_28_30.summary())

# 6. Group means for Google vs Perplexity
print("\n=== Group Means (Google vs Perplexity, Q28–Q30 usefulness scale) ===")
print(df_scale_28_30.groupby("condition")["tool_eval_q28_30_mean"].mean())


=== Cronbach's Alpha (Q28–Q30 usefulness scale) ===
0.8979854725015206

=== Head of Composite Scale Data (Q28–Q30) ===
   condition  q28_tool_info_provided  q29_tool_easier_task  q30_tool_useful  \
0          0                       5                     4                5   
1          0                       5                     5                5   
2          0                       6                     6                7   
3          0                       7                     7                7   
4          0                       7                     7                7   

   tool_eval_q28_30_mean  
0               4.666667  
1               5.000000  
2               6.333333  
3               7.000000  
4               7.000000  

=== OLS Regression Results (Q28–Q30 Composite) ===

                              OLS Regression Results                             
Dep. Variable:     tool_eval_q28_30_mean   R-squared:                       0.016
Model:                    

**Results on Perceived Usefulness of the Tool:**

1. **Reliability (Cronbach’s alpha, Q28–Q30):** The perceived usefulness scale based on Questions 28–30 (`q28_tool_info_provided`, `q29_tool_easier_task`, `q30_tool_useful`) showed good internal consistency (Cronbach’s alpha = 0.90). This indicates that the three items **coherently** measure participants’ perceived usefulness of the tool.

2. **Difference in perceived usefulness between Google and Perplexity:** An OLS regression predicting the composite usefulness score from tool condition (0 = Google, 1 = Perplexity) showed that participants tended to evaluate Google as more useful than Perplexity (b = -0.36, SE = 0.29, 95% CI [-0.93, 0.21]). However, this difference was **not statistically significant** (p = .213), so there is no clear evidence of a reliable usefulness advantage for either tool.

3. **Group means (Q28–Q30 usefulness scale):** On the 7‑point usefulness scale, participants rated Google slightly more positively (M = 5.12) than Perplexity (M = 4.76), a difference of about 0.36 points on average. This pattern is consistent with the negative regression coefficient but should be interpreted cautiously given the non‑significant p‑value.

### 3.4 Information Overload (Questions 31-32, q31_tool_too_much_info', 'q32_tool_difficult_focus')

In [23]:
# -------------------------------------------------------
# 3.4 Information Overload (Q31–Q32)
# -------------------------------------------------------

# 1. Specify items
items_31_32 = ["q31_tool_too_much_info", "q32_tool_difficult_focus"]
cols_31_32 = ["condition"] + items_31_32

# 2. Subset and drop rows with missing values in these items
df_scale_31_32 = df[cols_31_32].dropna()

# 3. Compute Cronbach's alpha (manual)
item_scores_31_32 = df_scale_31_32[items_31_32].to_numpy()

k_31_32 = item_scores_31_32.shape[1]  # number of items (2)
item_variances_31_32 = item_scores_31_32.var(axis=0, ddof=1)
total_score_variance_31_32 = item_scores_31_32.sum(axis=1).var(ddof=1)

alpha_31_32 = k_31_32/(k_31_32 - 1) * (1 - item_variances_31_32.sum() / total_score_variance_31_32)

print("\n=== Cronbach's Alpha (Q31–Q32 information overload scale) ===")
print(alpha_31_32)

# 4. Create composite scale (mean of Q31–Q32)
df_scale_31_32["info_overload_mean"] = df_scale_31_32[items_31_32].mean(axis=1)

print("\n=== Head of Composite Scale Data (Q31–Q32) ===")
print(df_scale_31_32.head())

# 5. Run OLS regression: composite ~ condition
model_scale_31_32 = smf.ols("info_overload_mean ~ condition", data=df_scale_31_32).fit()

print("\n=== OLS Regression Results (Q31–Q32 Information Overload Composite) ===\n")
print(model_scale_31_32.summary())

# 6. Group means for Google vs Perplexity
print("\n=== Group Means (Google vs Perplexity, Q31–Q32 information overload scale) ===")
print(df_scale_31_32.groupby("condition")["info_overload_mean"].mean())


=== Cronbach's Alpha (Q31–Q32 information overload scale) ===
0.8155782340365458

=== Head of Composite Scale Data (Q31–Q32) ===
   condition  q31_tool_too_much_info  q32_tool_difficult_focus  \
0          0                       2                       1.0   
1          0                       2                       2.0   
2          0                       3                       3.0   
3          0                       2                       2.0   
4          0                       5                       4.0   

   info_overload_mean  
0                 1.5  
1                 2.0  
2                 3.0  
3                 2.0  
4                 4.5  

=== OLS Regression Results (Q31–Q32 Information Overload Composite) ===

                            OLS Regression Results                            
Dep. Variable:     info_overload_mean   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                  0.023
Method:           

**Results on Information Overload:**

1. **Reliability (Cronbach’s alpha, Q31–Q32):** The information‑overload scale based on Questions 31–32 (`q31_tool_too_much_info`, `q32_tool_difficult_focus`) showed acceptable internal consistency (Cronbach’s alpha = 0.82). This indicates that the two items coherently capture participants’ experience of information overload while using the tool.

2. **Difference in perceived information overload between Google and Perplexity:** An OLS regression predicting the composite information‑overload score from tool condition (0 = Google, 1 = Perplexity) suggested that participants tended to experience more information overload with Perplexity than with Google (b = 0.60, SE = 0.33, 95% CI [-0.06, 1.26]). However, this effect was only marginally significant (p = .074) and did not reach conventional levels of statistical significance.

3. **Group means (Q31–Q32 information‑overload scale):** On the 7‑point information‑overload scale, participants reported lower overload for Google (M = 4.02) than for Perplexity (M = 4.62), a difference of about 0.60 points on average. This pattern is consistent with the positive regression coefficient and indicates a tendency for Perplexity to be experienced as more information‑overloading given the marginal p‑value.

### 3.5 Trust / Credibility of Results (Questions 33-35, 'q33_results_accurate', 'q34_results_trust', 'q35_results_complete',)

In [24]:
# -------------------------------------------------------
# 3.5 Perceived Result Quality (Q33–Q35)
# -------------------------------------------------------

# 1. Specify items
items_33_35 = ["q33_results_accurate", "q34_results_trust", "q35_results_complete"]
cols_33_35 = ["condition"] + items_33_35

# 2. Subset and drop rows with missing values in these items
df_scale_33_35 = df[cols_33_35].dropna()

# 3. Compute Cronbach's alpha (manual)
item_scores_33_35 = df_scale_33_35[items_33_35].to_numpy()

k_33_35 = item_scores_33_35.shape[1]  # number of items (3)
item_variances_33_35 = item_scores_33_35.var(axis=0, ddof=1)
total_score_variance_33_35 = item_scores_33_35.sum(axis=1).var(ddof=1)

alpha_33_35 = k_33_35/(k_33_35 - 1) * (1 - item_variances_33_35.sum() / total_score_variance_33_35)

print("\n=== Cronbach's Alpha (Q33–Q35 result quality scale) ===")
print(alpha_33_35)

# 4. Create composite scale (mean of Q33–Q35)
df_scale_33_35["result_quality_mean"] = df_scale_33_35[items_33_35].mean(axis=1)

print("\n=== Head of Composite Scale Data (Q33–Q35) ===")
print(df_scale_33_35.head())

# 5. Run OLS regression: composite ~ condition
model_scale_33_35 = smf.ols("result_quality_mean ~ condition", data=df_scale_33_35).fit()

print("\n=== OLS Regression Results (Q33–Q35 Result Quality Composite) ===\n")
print(model_scale_33_35.summary())

# 6. Group means for Google vs Perplexity
print("\n=== Group Means (Google vs Perplexity, Q33–Q35 result quality scale) ===")
print(df_scale_33_35.groupby("condition")["result_quality_mean"].mean())


=== Cronbach's Alpha (Q33–Q35 result quality scale) ===
0.8451610099517621

=== Head of Composite Scale Data (Q33–Q35) ===
   condition  q33_results_accurate  q34_results_trust  q35_results_complete  \
0          0                     4                  4                     2   
1          0                     4                  4                     4   
2          0                     5                  4                     3   
3          0                     6                  6                     6   
4          0                     5                  4                     4   

   result_quality_mean  
0             3.333333  
1             4.000000  
2             4.000000  
3             6.000000  
4             4.333333  

=== OLS Regression Results (Q33–Q35 Result Quality Composite) ===

                             OLS Regression Results                            
Dep. Variable:     result_quality_mean   R-squared:                       0.010
Model:                 

**Results on Trust / Credibility of Results:**

1. **Reliability (Cronbach’s alpha, Q33–Q35):** The result‑quality scale based on Questions 33–35 (`q33_results_accurate`, `q34_results_trust`, `q35_results_complete`) showed good internal consistency (Cronbach’s alpha = 0.85). This indicates that the three items **coherently capture** participants’ perceived accuracy, trustworthiness, and completeness of the results.

2. **Difference in perceived result quality between Google and Perplexity:** An OLS regression predicting the composite result‑quality score from tool condition (0 = Google, 1 = Perplexity) suggested that participants tended to evaluate Google’s results slightly more positively than Perplexity’s (b = -0.25, SE = 0.26, 95% CI [-0.77, 0.26]). However, this difference was **not statistically significant** (p = .333), so there is no clear evidence of a reliable advantage for either tool in perceived result quality.

3. **Group means (Q33–Q35 result‑quality scale):** On the 7‑point result‑quality scale, participants rated **Google marginally higher** (M = 4.84) than Perplexity (M = 4.59), a difference of about 0.25 points on average. but without significant results.

### 3.6 Relevance of Results (Questions 36-37, 'q36_results_relevant', 'q37_results_useful')

In [25]:
# -------------------------------------------------------
# 3.6 Relevance of Results (Q36–Q37)
# -------------------------------------------------------

# 1. Specify items
items_36_37 = ["q36_results_relevant", "q37_results_useful"]
cols_36_37 = ["condition"] + items_36_37

# 2. Subset and drop rows with missing values in these items
df_scale_36_37 = df[cols_36_37].dropna()

# 3. Compute Cronbach's alpha (manual)
item_scores_36_37 = df_scale_36_37[items_36_37].to_numpy()

k_36_37 = item_scores_36_37.shape[1]  # number of items (2)
item_variances_36_37 = item_scores_36_37.var(axis=0, ddof=1)
total_score_variance_36_37 = item_scores_36_37.sum(axis=1).var(ddof=1)

alpha_36_37 = k_36_37/(k_36_37 - 1) * (1 - item_variances_36_37.sum() / total_score_variance_36_37)

print("\n=== Cronbach's Alpha (Q36–Q37 relevance scale) ===")
print(alpha_36_37)

# 4. Create composite scale (mean of Q36–Q37)
df_scale_36_37["results_relevance_mean"] = df_scale_36_37[items_36_37].mean(axis=1)

print("\n=== Head of Composite Scale Data (Q36–Q37) ===")
print(df_scale_36_37.head())

# 5. Run OLS regression: composite ~ condition
model_scale_36_37 = smf.ols("results_relevance_mean ~ condition", data=df_scale_36_37).fit()

print("\n=== OLS Regression Results (Q36–Q37 Results Relevance Composite) ===\n")
print(model_scale_36_37.summary())

# 6. Group means for Google vs Perplexity
print("\n=== Group Means (Google vs Perplexity, Q36–Q37 relevance scale) ===")
print(df_scale_36_37.groupby("condition")["results_relevance_mean"].mean())


=== Cronbach's Alpha (Q36–Q37 relevance scale) ===
0.9273199579948048

=== Head of Composite Scale Data (Q36–Q37) ===
   condition  q36_results_relevant  q37_results_useful  results_relevance_mean
0          0                   5.0                 5.0                     5.0
1          0                   5.0                 5.0                     5.0
2          0                   6.0                 6.0                     6.0
3          0                   7.0                 7.0                     7.0
4          0                   5.0                 5.0                     5.0

=== OLS Regression Results (Q36–Q37 Results Relevance Composite) ===

                              OLS Regression Results                              
Dep. Variable:     results_relevance_mean   R-squared:                       0.005
Model:                                OLS   Adj. R-squared:                 -0.006
Method:                     Least Squares   F-statistic:                    0.4425
Date

**Results on Relevance of Results:**

1. **Reliability (Cronbach’s alpha, Q36–Q37):** The relevance scale based on Questions 36–37 (`q36_results_relevant`, `q37_results_useful`) showed excellent internal consistency (Cronbach’s alpha = 0.93). This indicates that the two items form a very **coherent scale** capturing how relevant and useful participants perceived the search results to be.

2. **Difference in perceived relevance between Google and Perplexity:** An OLS regression predicting the composite relevance score from tool condition (0 = Google, 1 = Perplexity) suggested that participants tended to rate Google’s results as slightly more relevant than Perplexity’s (b = -0.20, SE = 0.30, 95% CI [-0.79, 0.39]). However, this difference was **not statistically significant** (p = .508), so there is no clear evidence of a reliable relevance advantage for either tool.

3. **Group means (Q36–Q37 relevance scale):** On the 7‑point relevance scale, participants rated **Google’s results marginally higher** (M = 5.02) than Perplexity’s (M = 4.82), a difference of about 0.20 points on average. This small mean difference is in line with the negative regression coefficient but should be interpreted with caution given the non‑significant p‑value.